In [ ]:
# default_exp problem_types.vector_fit
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# test setup
import tensorflow as tf
import numpy as np
from m3tl.test_base import TestBase
from m3tl.input_fn import train_eval_input_fn
from m3tl.test_base import test_top_layer
test_base = TestBase()
params = test_base.params

hidden_dim = params.bert_config.hidden_size

train_dataset = train_eval_input_fn(params=params)
one_batch = next(train_dataset.as_numpy_iterator())


Adding new problem weibo_fake_ner, problem type: seq_tag
Adding new problem weibo_cws, problem type: seq_tag
Adding new problem weibo_fake_multi_cls, problem type: multi_cls
Adding new problem weibo_fake_cls, problem type: cls
Adding new problem weibo_masklm, problem type: masklm
Adding new problem weibo_pretrain, problem type: pretrain
Adding new problem weibo_fake_regression, problem type: regression
Adding new problem weibo_fake_vector_fit, problem type: vector_fit


/data/m3tl/m3tl/read_write_tfrecord.py:83: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif np.issubdtype(type(feature), np.float):


INFO:tensorflow:sampling weights: 
INFO:tensorflow:weibo_fake_cls_weibo_fake_ner_weibo_fake_regression_weibo_fake_vector_fit: 0.6666666666666666
INFO:tensorflow:weibo_fake_multi_cls: 0.16666666666666666
INFO:tensorflow:weibo_masklm: 0.16666666666666666


# VectorFit(vector_fit)

This module includes neccessary part to register vector_fit problem type.

## Imports and utils


In [ ]:
# export
from typing import Dict, Tuple

import numpy as np
import tensorflow as tf
from m3tl.base_params import BaseParams
from m3tl.problem_types.utils import (empty_tensor_handling_loss,
                                      nan_loss_handling)
from m3tl.special_tokens import PREDICT
from m3tl.utils import get_phase


## Top Layer

In [ ]:
# export
def cosine_wrapper(labels, logits, from_logits=True):
    return tf.keras.losses.cosine_similarity(labels, logits)


class VectorFit(tf.keras.Model):
    def __init__(self, params: BaseParams, problem_name: str) -> None:
        super(VectorFit, self).__init__(name=problem_name)
        self.params = params
        self.problem_name = problem_name
        self.num_classes = self.params.get_problem_info(problem=problem_name, info_name='num_classes')
        self.dense = tf.keras.layers.Dense(self.num_classes)

    def call(self, inputs: Tuple[Dict]):
        mode = get_phase()
        feature, hidden_feature = inputs
        pooled_hidden = hidden_feature['pooled']

        logits = self.dense(pooled_hidden)
        if mode != PREDICT:
            # this is actually a vector
            label = feature['{}_label_ids'.format(self.problem_name)]

            loss = empty_tensor_handling_loss(label, logits, cosine_wrapper)
            loss = nan_loss_handling(loss)
            self.add_loss(loss)

            self.add_metric(tf.math.negative(
                loss), name='{}_cos_sim'.format(self.problem_name), aggregation='mean')
        return logits

In [ ]:
test_top_layer(VectorFit, problem='weibo_fake_vector_fit', params=params, sample_features=one_batch, hidden_dim=hidden_dim)

Testing VectorFit


## Get or make label encoder function


In [ ]:
# export
def vector_fit_get_or_make_label_encoder_fn(params: BaseParams, problem, mode, label_list, *args, **kwargs):
    if label_list:
        # set params num_classes for this problem
        label_array = np.array(label_list)
        params.set_problem_info(problem=problem, info_name='num_classes', info=label_array.shape[-1])
    return None


## Label handing function

In [ ]:
# export
def vector_fit_label_handling_fn(target, label_encoder=None, tokenizer=None, decoding_length=None, *args, **kwargs):
    # return label_id and label mask
    label_id = np.array(target, dtype='float32')
    return label_id, None

